# 中移集成（雄安产业研究院）首届OneCity编程大赛


竞赛地址：
https://www.dcjingsai.com/v2/cmptDetail.html?id=457

参考资料地址：
https://www.kesci.com/mw/project/5be7e948954d6e0010632ef2

In [1]:
import zipfile
import os
import pandas as pd
import jieba
import numpy as np

In [2]:
Datasets_PATH = '~/OneDrive/ProjectSpace/Jupyter/datasets/'
TrainSets_filename = 'answer_train.csv'
TestSets_filename = 'submit_example_test1.csv'

In [3]:
Raw_Data = pd.read_csv(Datasets_PATH+TrainSets_filename)

In [4]:
Raw_Data

,filename,label
0,train/市管宗教活动场所.csv,文化休闲
1,train/南涧彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,医疗卫生
2,train/价格监测信息公开事项汇总信息_.xls,经济管理
3,train/县级公共图书馆文化馆总分馆制信息_大安市.csv,文化休闲
4,train/县级公共图书馆文化馆总分馆制信息_陇南市武都区.csv,文化休闲
...,...,...
59995,train/二季度行政处置状况公告信息_.xls,文秘行政
59996,train/社保_机关事业单位离退休人员及离休遗属增减，离退休费、离休人员抚恤金、丧葬费及遗...,劳动人事
59997,train/托里县农林牧渔业产值.csv,农业畜牧业
59998,train/设立经营性互联网文化单位审批_忻州市.csv,文化休闲


In [5]:
Raw_Data.label.unique()

array(['文化休闲', '医疗卫生', '经济管理', '教育科技', '城乡建设', '工业', '民政社区', '交通运输',
       '生态环境', '政法监察', '农业畜牧业', '文秘行政', '劳动人事', '资源能源', '信息产业', '旅游服务',
       '商业贸易', '气象水文测绘地震地理', '财税金融', '外交外事'], dtype=object)

In [6]:
jieba.enable_parallel() #并行分词开启
Raw_Data['文本分词'] = Raw_Data['filename'].apply(lambda i:jieba.cut(i) )

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.168 seconds.
Prefix dict has been built successfully.


In [7]:
Raw_Data['文本分词'] =[' '.join(i) for i in Raw_Data['文本分词']]

In [8]:
Raw_Data.head()

,filename,label,文本分词
0,train/市管宗教活动场所.csv,文化休闲,train / 市管 宗教 活动场所 . csv
1,train/南涧彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,医疗卫生,train / 南涧彝族自治县 市 汶上县 残疾人 联合会 _ 残疾人 证 办理 _. csv
2,train/价格监测信息公开事项汇总信息_.xls,经济管理,train / 价格 监测 信息 公开 事项 汇总 信息 _. xls
3,train/县级公共图书馆文化馆总分馆制信息_大安市.csv,文化休闲,train / 县级 公共 图书馆 文化馆 总分 馆制 信息 _ 大安市 . csv
4,train/县级公共图书馆文化馆总分馆制信息_陇南市武都区.csv,文化休闲,train / 县级 公共 图书馆 文化馆 总分 馆制 信息 _ 陇南 市 武都 区 . csv


In [9]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """对数损失度量（Logarithmic Loss  Metric）的多分类版本。
    :param actual: 包含actual target classes的数组
    :param predicted: 分类预测结果矩阵, 每个类别都有一个概率
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [10]:
from sklearn import preprocessing

In [11]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(Raw_Data.label.values)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Raw_Data.文本分词.values, y, 
                                                  stratify=y,
                                                  random_state=55, 
                                                  test_size=0.1, shuffle=True)
print (xtrain.shape)
print (xvalid.shape)

(54000,)
(6000,)


In [14]:
xtrain

array(['train / 盘锦市 盘山县 民声 阅办 记录 信息 _. csv',
       'train / 长岛 国家级 自然保护区 管理局 年 新闻宣传 状况 总括 表 _. xls',
       'train / 淮安市 旅行社 目录 . csv', ...,
       'train / 新宾满族自治县 县 法人 行政处罚 数据 . csv',
       'train / 太原市 杏花岭区 市 广东省 名牌产品 工业 类 称号 信息 _. xls',
       'train / 山西省 长治市 黎城县 县 法人 行政处罚 数据 . csv'], dtype=object)

In [15]:
def number_normalizer(tokens):
    """ 将所有数字标记映射为一个占位符（Placeholder）。
    对于许多实际应用场景来说，以数字开头的tokens不是很有用，
    但这样tokens的存在也有一定相关性。 通过将所有数字都表示成同一个符号，可以达到降维的目的。
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [17]:
class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super(NumberNormalizingVectorizer, self).build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))

利用刚才创建的NumberNormalizingVectorizer类来提取文本特征，注意里面各类参数的含义，自己去sklearn官方网站找教程看

stwlist=[line.strip() for line in open('/home/kesci/input/stopwords7085/停用词汇总.txt',
                                       'r',encoding='utf-8').readlines()]

tfv = NumberNormalizingVectorizer(min_df=3,  
                                  max_df=0.5,
                                  max_features=None,                 
                                  ngram_range=(1, 2), 
                                  use_idf=True,
                                  smooth_idf=True,
                                  stop_words = stwlist)

In [18]:
# 利用刚才创建的NumberNormalizingVectorizer类来提取文本特征，注意里面各类参数的含义，自己去sklearn官方网站找教程看
stwlist=[line.strip() for line in open('/home/zhuangbin/OneDrive/ProjectSpace/Jupyter/datasets/baidu_stopwords.txt','r',encoding='utf-8').readlines()]

tfv = NumberNormalizingVectorizer(min_df=3,  
                                  max_df=0.5,
                                  max_features=None,                 
                                  ngram_range=(1, 2), 
                                  use_idf=True,
                                  smooth_idf=True,
                                  stop_words = stwlist)

In [19]:
from sklearn.linear_model import LogisticRegression

In [22]:
# 使用TF-IDF来fit训练集和测试集（半监督学习）
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

# 使用TF-IDF来fit训练集和测试集（半监督学习）
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

/home/zhuangbin/pyenv/jupyter/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


NumberNormalizingVectorizer(max_df=0.5, min_df=3, ngram_range=(1, 2),
                            stop_words=['--', '?', '“', '”', '》', '－－', 'able',
                                        'about', 'above', 'according',
                                        'accordingly', 'across', 'actually',
                                        'after', 'afterwards', 'again',
                                        'against', "ain't", 'all', 'allow',
                                        'allows', 'almost', 'alone', 'along',
                                        'already', 'also', 'although', 'always',
                                        'am', 'among', ...])

NumberNormalizingVectorizer(max_df=0.5, min_df=3, ngram_range=(1, 2),
                            stop_words=['--', '?', '“', '”', '》', '－－', 'able',
                                        'about', 'above', 'according',
                                        'accordingly', 'across', 'actually',
                                        'after', 'afterwards', 'again',
                                        'against', "ain't", 'all', 'allow',
                                        'allows', 'almost', 'alone', 'along',
                                        'already', 'also', 'although', 'always',
                                        'am', 'among', ...])

In [ ]:
#利用提取的TFIDF特征来fit一个简单的Logistic Regression 
clf = LogisticRegression(C=1.0,solver='lbfgs',multi_class='multinomial')
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))
#print(classification_report(predictions, yvalid))

In [26]:
ctv = CountVectorizer(min_df=3,
                      max_df=0.5,
                      ngram_range=(1,2),
                      stop_words = stwlist)

# 使用Count Vectorizer来fit训练集和测试集（半监督学习）
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

/home/zhuangbin/pyenv/jupyter/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CountVectorizer(max_df=0.5, min_df=3, ngram_range=(1, 2),
                stop_words=['--', '?', '“', '”', '》', '－－', 'able', 'about',
                            'above', 'according', 'accordingly', 'across',
                            'actually', 'after', 'afterwards', 'again',
                            'against', "ain't", 'all', 'allow', 'allows',
                            'almost', 'alone', 'along', 'already', 'also',
                            'although', 'always', 'am', 'among', ...])

In [ ]:
#利用提取的word counts特征来fit一个简单的Logistic Regression 
clf = LogisticRegression(C=1.0,solver='lbfgs',multi_class='multinomial')
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

#print(classification_report(predictions, yvalid))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
#利用提取的TFIDF特征来fitNaive Bayes
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

#print(classification_report(predictions, yvalid))

In [ ]:
#利用提取的word counts特征来fitNaive Bayes
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

#print(classification_report(predictions, yvalid))

In [ ]:
from sklearn import decomposition

In [ ]:
#使用SVD进行降维，components设为120，对于SVM来说，SVD的components的合适调整区间一般为120~200 
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

#对从SVD获得的数据进行缩放
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [ ]:
from sklearn.svm import SVC

In [ ]:
# 调用下SVM模型
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

#print(classification_report(predictions, yvalid))

In [23]:
import xgboost as xgb

In [24]:
# 基于tf-idf特征，使用xgboost
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

#print(classification_report(predictions, yvalid))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=10, nthread=10, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

logloss: 0.301 


In [27]:
# 基于word counts特征，使用xgboost
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

#print(classification_report(predictions, yvalid))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=10, nthread=10, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

logloss: 0.304 


In [ ]:
Raw_Test = pd.read_csv(Datasets_PATH+TestSets_filename)

In [ ]:
jieba.enable_parallel() #并行分词开启
Raw_Test['文本分词'] = Raw_Test['filename'].apply(lambda i:jieba.cut(i) )

In [ ]:
Raw_Test['文本分词'] =[' '.join(i) for i in Raw_Test['文本分词']]

In [ ]:
Raw_Test['文本分词']

In [ ]:
X_test_ctv =  ctv.transform(Raw_Test['文本分词']) 

In [ ]:
y_test = clf.predict(X_test_ctv.tocsc())

In [ ]:
y_test

In [ ]:
y_test_rlt = lbl_enc.inverse_transform(y_test)

In [ ]:
y_test_rlt.tofile()

In [ ]:
pd.DataFrame(y_test_rlt).to_csv(Datasets_PATH+'rlt_onecity_001.csv')

In [ ]:
Raw_Test['result'] = pd.DataFrame(y_test_rlt)

In [ ]:
Raw_Test.to_csv(Datasets_PATH+'submit_result_201113_01.csv',)

In [ ]:
Raw_Test[['filename','result']]